In [1]:
# Import the two main libraries
from bs4 import BeautifulSoup       # to process html
import requests
import urllib.request,sys,time
import pandas as pd
from tqdm import tqdm
import pickle
import datetime

In [142]:
import warnings
warnings.filterwarnings("ignore")

This notebook is a fork of the one [about the Sun](./scraper_sun.ipynb) for more information look at that one before.

In [17]:
#get all the links of the articles

saved_links = []
for page in tqdm(range(1,30)):

    url = 'https://www.bbc.co.uk/search?q=russia-ukraine+war&d=news_gnl&page='+str(page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all('a',{'ssrcss-rl2iw9-PromoLink e1f5wbog1'})
    
    for link in links:
        saved_links.append(link['href'].strip())

100%|██████████| 29/29 [00:05<00:00,  5.37it/s]


In [1]:
#We put the link into the pickle file
file_name = "bbc_links.pkl"

open_file = open(file_name, "wb") #writte on the pickle file
pickle.dump(saved_links, open_file)
open_file.close()

NameError: name 'pickle' is not defined

In [2]:
# to load
open_file = open("bbc_links.pkl", "rb") #read the pickle file 
saved_links = pickle.load(open_file)
open_file.close()

In [7]:
#creating the dataframe

BbcArticles = pd.DataFrame({'title': pd.Series([], dtype='string'),
                             'author': pd.Series([], dtype='string'),
                             'date': pd.Series([], dtype='float'),
                             'text': pd.Series([], dtype='string'),
                             })


The main difference with the one of the Sun is that not all the link that we have of the BBC are about articles, so sometimes beautiful soup fail while is searching f

In [11]:
for i,link in tqdm(enumerate(saved_links)):
    
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    try: # the try except is made to handle the problem that some links of the bbc aren't article, so when beautiful soup fail to load the article tag the news is skipped
        # get article tag
        article = soup.find_all("article")[0]

        #get title
        header = article.find_all("header")[0]
        title = header.find_all("h1")[0].get_text()

        #get author
        author = article.find('div', {'class': 'ssrcss-68pt20-Text-TextContributorName'})
        if author == None:
            author = article.find('span', {'class': 'qa-contributor-name'}).get_text()
        else:
            author = author.get_text()

        #get date
        date = article.time['datetime']
        date = pd.to_datetime(date).strftime('%d/%m/%y')

        #get text
        text = ''
        for p in article.find_all('p'):
            text += p.text

        BbcArticles = BbcArticles.append({"title": title, "author": author, "date": date, "text": text}, ignore_index=True)
    except:
        pass

290it [00:52,  5.55it/s]


In [12]:
BbcArticles

,title,author,date,text
0,Ukraine war: Russia launches second pre-dawn m...,By Hugo Bachega in Kyiv & Antoinette Radford i...,01/05/23,Russia has launched a series of missiles at Uk...
1,Ukraine war: 25 dead as Russian missiles hit c...,By Hugo Bachega in Uman & Antoinette Radford i...,28/04/23,A wave of Russian air strikes on cities across...
2,Ukraine war: The Nigerian paying a 'heavy pric...,By Oluwashina Okeleji,13/04/23,Last updated on 13 April 202313 April 2023.Fro...
3,Ukraine war: Partisan attack kills police chie...,By Robert Greenall,27/04/23,A police chief in the occupied Ukrainian city ...
4,Ukraine war: Where are Russia's opposition lea...,By Vitaly Shevchenko,17/04/23,President Vladimir Putin now rules Russia virt...
...,...,...,...,...
345,Ukraine war: Drone attack on Russian bomber ba...,By Russia Editor Steve Rosenberg & Jaroslav Lukiv,26/12/22,A Ukrainian drone attack on an airbase for bom...
346,Ukraine war: Zelensky says Russia controls a f...,By Matt Murphy,02/06/22,Ukraine's President Volodymyr Zelensky says th...
347,Ukraine war: The deadly landmines killing hund...,By James Waterhouse,11/04/23,"Across Ukraine's vast expanse, there are thoug..."
348,Ukraine war: Russia's jailed Navalny attacks i...,By Paul Kirby,24/05/22,This video can not be playedWatch: Alexei Nava...


In [10]:
BbcArticles.to_csv("BbcArticles.csv")